In [2]:
# The usual fare
# Importing
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline
from astropy.io import fits
import matplotlib.cm as cm
import os 
import glob 
import astropy
import scipy.signal
from astropy.stats import sigma_clip
import time
from astropy import stats
from PIL import Image
import shift_methods as sm
import textwrap as tw

In [3]:
def mediancombine(filelist):
    '''
    Edit this docstring accordingly!
    '''
    n = len(filelist)
    first_frame_data = fits.getdata(filelist[0])
    imsize_y, imsize_x = first_frame_data.shape
    fits_stack = np.zeros((imsize_y, imsize_x , n), dtype = np.float32) 
    for ii in range(0, n):
        im = fits.getdata(filelist[ii])
        fits_stack[:,:,ii] = im
    med_frame = np.median(fits_stack, axis=2)
    return med_frame
# still need to median_bias = mediancombine(biasfiles)
# fits.writeto('Backup_Master_Bias.fit', median_bias, biasheader, clobber=True)
# Make Master Bias Path

In [4]:
def filesorter(filename, foldername, fitskeyword_to_check, keyword):
    '''
    This will sort your observation data files for you. It takes in the name of a file,
    the folder you want it in, the keyword in fits to look for, and the keyword category
    in fits to look for the keyword in. Ex. sorting calibrations into new folder --
    filesorter(filename,'calibration','cal','OBJECT')
    '''
    if os.path.exists(filename):
        pass
    else:
        print(filename + " does not exist or has already been moved.")
        return
    
    header = fits.getheader(filename)
    fits_type = header[keyword]
    
    if os.path.exists(foldername):
        pass
    else:
        print("Making new directory: " + foldername)
        os.mkdir(foldername)
        
    if fits_type == fitskeyword_to_check:
        destination = foldername + '/'
        print("Moving " + filename + " to: ./" + destination + filename)
        os.rename(filename, destination + filename)  

    return

In [5]:
def FileMover(filelist,foldername):
        '''
    do this docstring you asshole
    '''
        for file in filelist:
            if os.path.exists(file):
                pass
            else:
                print(file + "does not exist or has already been moved")
                return
            if os.path.exists(foldername):
                pass
            else:
                print("Making new directory:" + foldername)
                os.mkdir(foldername)
            destination = foldername + '/'
            print("Moving " + file + " to: ./" + destination + file)
            os.rename(file, destination + file)  


In [6]:
def CustomBias(filelist):
    '''
    do this docstring you asshole
    '''
    for file in filelist:
        im = fits.getdata(file)
        new_name = 'b_'+file[6:-7]+'.fits'
        if im.shape == (4150,4150):
            #im_name = file[6:-7]
            overscan = np.mean(im[:,4100:4140])
            custom_bias = overscan * mmaster_bias
            bias_subtracted = im - custom_bias
            bias_sub_overscan = bias_subtracted[:4110,:4090]
            header = fits.getheader(filelist[0])
            fits.writeto(new_name,bias_sub_overscan, header,overwrite=True)
            print(new_name,'written.')

In [7]:
def masterflat(filelist,filt,flatdec,filtnum):
    '''
    do this docstring you asshole
    '''
    allimages=filelist
    filt=filt
    flat_images = []
    for flat in allimages:
        if fits.getheader(flat)['DECSTRNG']==flatdec and fits.getheader(flat)['FILTER2 ']==filtnum:
            flat_images.append(flat)
    print(flat_images)
    try:
        flat_header= fits.getheader(flat_images[0])
        CustomBias(flat_images)
        FileMover(flat_images,filt+'Flat')
        b_flat_images = glob.glob('b_*.fits')
        Master_Flat=norm_combine_flats(b_flat_images)
        fits.writeto('Master'+filt+'Flat.fits',Master_Flat,flat_header,overwrite=True)
        print('Master'+filt+'Flat Image Created')
        FileMover(b_flat_images,filt+'_bflats')
    except IndexError:
        print("""Couldn\'t find any files. You might not have any data for this filter on this night. Check your logs.""")
        
    return Master_Flat

In [8]:
def flatfield(filelist,masterflat):
    '''
    do this docstring you asshole
    '''
    filelist=filelist
    masterflat = masterflat
    print('Starting flatfielding')
    print('*'*25)
    for file in filelist:
        data = fits.getdata(file)
        header=fits.getheader(file)
        flatfielded = data / masterflat
        filename = 'f_'+file
        fits.writeto(filename,flatfielded,header,overwrite=True)
        print(file, 'has been flatfielded')
        print('*'*25)
    print('Done flatfielding')

In [9]:
def norm_combine_flats(filelist):
    '''
    This normalizes the flat field images. Each flat will have a 
    pixel value of ~1. Once we do this, we can then median combine
    the flats to get our master flat for a given filter
    '''
    n = len(filelist)
    first_frame_data = fits.getdata(filelist[0])
    imsize_y, imsize_x = first_frame_data.shape
    fits_stack = np.zeros((imsize_y, imsize_x , n), dtype = np.float32) 
    for ii in range(0, n):
        im = fits.getdata(filelist[ii])
        norm_im =  im/np.median(im)# finish new line here to normalize flats
        fits_stack[:,:,ii] = norm_im
    med_frame = np.median(fits_stack, axis=2)
    return med_frame

In [10]:
def imageshifter(imlist):
    '''
    do this docstring you asshole
    '''
    imlist=imlist
    imlist.sort()
    # i=0
    # worklist= imlist[i:i+3]
    # for i in range(len(imlist)):
    im1,hdr1 = fits.getdata(imlist[0],header=True)
    #     # print(imlist[0])
    xshifts={}
    yshifts={}
    shift_image={}

    for index,filename in enumerate(imlist):
        imstack = np.zeros([im1.shape[0],im1.shape[1],len(imlist)])
        im = fits.getdata(filename)
        xshifts[index],yshifts[index]=sm.cross_image(im1,im)
        shift_image[index] = sm.shift_image(im,xshifts[index],yshifts[index])
        name = 's_'+filename
        fits.writeto(name,shift_image[index],hdr1,overwrite=True)
        print("Shift for image",name,"is",xshifts[index],yshifts[index])
        print('FITS file written:',name)
        print('*'*25)
    print('Stacking done!')

In [21]:
def ministack(imlist,obj,filt):
    imlist=shifted
    im1 = fits.getdata(shifted[0])
    i=0
    obj = obj
    filt = filt
    worklist=shifted[i:i+3]
    print('Starting Ministacks')
    print('*'*25)
    for i in range(len(shifted)):
        if worklist != []: 
            medianimage = mediancombine(worklist)
            num = str(i)
            header = fits.getheader(worklist[0])
            fits.writeto(num+'_'+obj+'_'+filt+'_stack.fits',medianimage,header,overwrite=True)
            print('Wrote FITS File:',num+'_'+obj+'_'+filt+'_stack.fits')
            print('*'*25)

        else:
            break

        i+=i*i+3
        worklist=imlist[i:i+3]
    print('Done Ministacking')

In [12]:
def masterbias(directory,all_images):
    for file in all_images:
        filesorter(file,'bias',0.001,'EXPTIME')
    biasdir = directory+'/bias'
    os.chdir(biasdir)
    print('Moving into',biasdir)
    bias_images = glob.glob('*')
    if bias_images ==[]:
        print(tw.fill('No biases found. Check your logs. You might need to use the nearest night\'s bias stack',50))
        biasdir = input('Please type the file path to the nearest bias data')
        os.chdir(biasdir)
        new_ims=glob.glob('*.fits')
        for file in new_ims:
            filesorter(file,'bias',0.001,'EXPTIME')
        os.chdir(biasdir+'/bias')
        new_biases = glob.glob('*')
        print(new_biases)
        masterbias = mediancombine(new_biases)
        biasheader = fits.getheader(new_biases[0])
        
    else: 
        masterbias = mediancombine(bias_images)
        biasheader = fits.getheader(bias_images[0])
    mmaster_bias = masterbias/np.median(masterbias)
    fits.writeto('MasterBias.fits',mmaster_bias,biasheader,overwrite=True)
    print('MasterBias.fits generated')
    print('Master Bias array generated')
    return mmaster_bias

In [13]:
def scifinder(directory,obj,objdec,filtnum):
    os.chdir(directory)
    all_images = glob.glob('*.fits')
    ims = []
    for im in all_images:
        if fits.getheader(im)['DECSTRNG']==objdec and fits.getheader(im)['FILTER2 ']==filtnum:
            ims.append(im)
    print('I found',len(ims),obj,'images in filter number',filtnum)
    CustomBias(ims)
    FileMover(ims,obj+'_'+filt+'_science')
    ims_b = glob.glob('b_*.fits')
    FileMover(ims_b,'b_'+obj+'_'+filt+'_science')

In [31]:
os.chdir(directory)
all_images=glob.glob('*fits')
mmaster_bias = masterbias(directory,all_images)
os.chdir(directory)
all_images=glob.glob('*fits')
masterflat(all_images,filt,'+13:00:03','200')


In [71]:
V410dec = '+28:24:51'
HLTaudec= '+18:14:25'
IC348dec='+32:03:34' 


In [48]:
obj = 'HL Tau'
filt= 'Ha'
# Master Bias Array and FITS file generation
directory = input('Where can I find your bias files? ')
os.chdir(directory)
all_images=glob.glob('*.fits')
for file in all_images:
    filesorter(file,'bias',0.001,'EXPTIME') # get the biases
os.chdir(directory+'/bias')
bias_images=glob.glob('*') # gather bias images  
try:
    first_bias_data = fits.getdata(bias_images[0]) #
    n = len(bias_images) # finds how man bias frames there are so 
                             # we can make the right sized array for them
                             # to stack
    imsize_y, imsize_x = first_bias_data.shape
                             # gets the size of the image to make the
                             # right sized array for all the pixel values
    biasarray = np.zeros((imsize_y, imsize_x , n), dtype = np.float32)
                         # make an empty array for all our stuff

    for ii in range(0, n):
        im = fits.getdata(bias_images[ii]) # gets the image data...
        biasarray[:,:,ii] = im # and adds it to a layer of the array


    biasheader = fits.getheader(bias_images[0]) # gets fits header for bias frames
    master_bias = mediancombine(bias_images) # performs median combo
    mmaster_bias = master_bias/np.mean(master_bias) # normalizes the master bias for overscan scaling later
    print('Normalized Master Bias array generated')
    fits.writeto('Norm_Master_Bias.fits',mmaster_bias,biasheader,overwrite=True)
    print('Norm_Master_Bias.fits generated')
    os.chdir(directory) 
except IndexError:
    print('No biases found here.')

What directory of data do you want to work with? E:\20190124
Moving c8506t0398b00.fits to: ./bias/c8506t0398b00.fits
Moving c8506t0399b00.fits to: ./bias/c8506t0399b00.fits
Moving c8506t0400b00.fits to: ./bias/c8506t0400b00.fits
Moving c8506t0401b00.fits to: ./bias/c8506t0401b00.fits
Moving c8506t0402b00.fits to: ./bias/c8506t0402b00.fits
Moving c8506t0403b00.fits to: ./bias/c8506t0403b00.fits
Moving c8506t0404b00.fits to: ./bias/c8506t0404b00.fits
Moving c8506t0405b00.fits to: ./bias/c8506t0405b00.fits
Moving c8506t0406b00.fits to: ./bias/c8506t0406b00.fits
Moving c8506t0407b00.fits to: ./bias/c8506t0407b00.fits
Moving c8507t0040b00.fits to: ./bias/c8507t0040b00.fits
Moving c8506t0397b00.fits to: ./bias/c8506t0397b00.fits
Normalized Master Bias array generated
Norm_Master_Bias.fits generated


FileNotFoundError: [Errno 2] No such file or directory: 'c8506t0398b00.fits'

In [65]:
# Master Flat generator
directory = input('Where can I find your dome flats?')
os.chdir(directory)
all_images = glob.glob('c*.fits')
MasterFlat = masterflat(all_images,'Ha','+13:00:03','200')
#all_images = glob.glob('*.fits')
os.chdir(directory)
print('Moving back to '+directory)

Where can I find your dome flats?E:\20190122\HaFlat
['c8504t0163f00.fits', 'c8504t0164f00.fits', 'c8504t0165f00.fits', 'c8504t0166f00.fits', 'c8504t0167f00.fits']
b_0163f.fits written.
b_0164f.fits written.
b_0165f.fits written.
b_0166f.fits written.
b_0167f.fits written.
Making new directory:HaFlat
Moving c8504t0163f00.fits to: ./HaFlat/c8504t0163f00.fits
Moving c8504t0164f00.fits to: ./HaFlat/c8504t0164f00.fits
Moving c8504t0165f00.fits to: ./HaFlat/c8504t0165f00.fits
Moving c8504t0166f00.fits to: ./HaFlat/c8504t0166f00.fits
Moving c8504t0167f00.fits to: ./HaFlat/c8504t0167f00.fits
MasterHaFlat Image Created
Making new directory:Ha_bflats
Moving b_0163f.fits to: ./Ha_bflats/b_0163f.fits
Moving b_0164f.fits to: ./Ha_bflats/b_0164f.fits
Moving b_0165f.fits to: ./Ha_bflats/b_0165f.fits
Moving b_0166f.fits to: ./Ha_bflats/b_0166f.fits
Moving b_0167f.fits to: ./Ha_bflats/b_0167f.fits


In [1]:
directory = input('Where can I find some science images?')
# os.chdir(directory)
scifinder(directory,'HLTau',HLTaudec,'200')
# biassub = glob.glob('b_*')
# flatfield(biassub,MasterFlat)
# flatfielded = glob.glob('f_b_*')
# imageshifter(flatfielded)
# all_images=glob.glob('*.fits')
# all_images
# ims = []
# for im in all_images:
#     if fits.getheader(im)['DECSTRNG']==HLTaudec and fits.getheader(im)['FILTER2 ']=='200':
#         ims.append(im)
#         print(ims)
# os.chdir(directory)

# filtnum = '200'
# objdec = HLTaudec
# obj = 'HL Tau'
# all_images = glob.glob('*.fits')
# ims = []
# for im in ims:
#     if fits.getheader(im)['DECSTRNG']==objdec and fits.getheader(im)['FILTER2 ']==filtnum:
#         ims.append(im)
# print('I found',len(ims),obj,'images in filter number',filtnum)

Where can I find some science images?E:\20190112


NameError: name 'scifinder' is not defined

In [97]:
os.chdir(directory+'/b_'+obj+'_'+filt+'_science')
b_sciims=glob.glob('b_*')
flatfield(b_sciims,MasterFlat)

Starting flatfielding
*************************
b_0110o.fits has been flatfielded
*************************
b_0112o.fits has been flatfielded
*************************
b_0111o.fits has been flatfielded
*************************
Done flatfielding


In [17]:
directory = 'E:/20190122/b_HL Tau_Ha_science'
os.chdir(directory)
flatfielded = glob.glob('f_b_*')
imageshifter(flatfielded)
shifted = glob.glob('s_*')
ministack(shifted,'HLTau','Ha')

Shift for image s_f_b_0110o.fits is 0.0 0.0
FITS file written: s_f_b_0110o.fits
*************************
Shift for image s_f_b_0111o.fits is 0.2816938290275175 1.1856918281289381
FITS file written: s_f_b_0111o.fits
*************************
Shift for image s_f_b_0112o.fits is -0.0991369429475526 -0.04191010767704029
FITS file written: s_f_b_0112o.fits
*************************
Stacking done!
Starting Ministacks
*************************


NameError: name 'num' is not defined

In [20]:
shifted = glob.glob('s_*')
header=fits.getheader(shifted[0])
ministack(shifted,'HLTau','Ha')

Starting Ministacks
*************************
Wrote FITS File: 0_HLTau_Ha_stack.fits
*************************
Done Ministacking


In [100]:
ministack?

In [ ]:
cd 20190113/

In [ ]:
all_images=glob.glob('*.fits')
for file in all_images:
    filesorter(file,'bias',0.001,'EXPTIME') # get the biases

In [ ]:
cd bias/

In [ ]:
all_images=glob.glob('*.fits')
for file in all_images:
    filesorter(file,'bias',0.001,'EXPTIME') # get the biases
os.chdir('\bias\')
bias_images=glob.glob('*') # gather bias images
first_bias_data = fits.getdata(bias_images[0]) #
n = len(bias_images) # finds how man bias frames there are so 
                     # we can make the right sized array for them
                     # to stack
imsize_y, imsize_x = first_bias_data.shape
                     # gets the size of the image to make the
                     # right sized array for all the pixel values
biasarray = np.zeros((imsize_y, imsize_x , n), dtype = np.float32)
                     # make an empty array for all our stuff
    
for ii in range(0, n):
    im = fits.getdata(bias_images[ii]) # gets the image data...
    biasarray[:,:,ii] = im # and adds it to a layer of the array
    

biasheader = fits.getheader(bias_images[0]) # gets fits header for bias frames
master_bias = mediancombine(bias_images) # performs median combo
mmaster_bias = master_bias/np.mean(master_bias) # normalizes the master bias for overscan scaling later
print('Normalized Master Bias array generated')

In [ ]:
os.chdir('E:/20190113')

In [ ]:
imlist=glob.glob('*.fits')
masterflat(imlist,'Ha','+13:00:03','200')


In [ ]:
os.chdir('E:/20190112')

In [ ]:
master_haflat = fits.getdata(os.getcwd()+"\MasterHaFlat.fits")

In [ ]:
allims=glob.glob('*.fits')
allims.sort()
allims

masterflat(allims,'Ha','+13:00:03','200')